# 🏦 Banco Insights 2.0 - Metrics Calculations Documentation

**Created**: January 2025  
**Purpose**: Comprehensive documentation and validation of all financial metrics calculations  
**Audience**: FSI professionals, auditors, developers, and compliance teams  
**Standards**: BACEN regulations and Basel framework compliance

---

## 📋 Table of Contents

1. [Setup and Data Import](#setup)
2. [Simple Aggregations](#simple-aggregations)  
3. [Complex Financial Ratios](#complex-ratios)
4. [Market Analysis Calculations](#market-analysis)
5. [Statistical Aggregations](#statistical-aggregations)
6. [Validation and Quality Assurance](#validation)
7. [Examples and Use Cases](#examples)
8. [Compliance and Audit Trail](#compliance)

---

## 1. Setup and Data Import {#setup}

**IMPORTANT**: This notebook demonstrates calculations without opening the large consolidated_cleaned.csv file directly to avoid memory issues.

In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
import json
from typing import Dict, List, Tuple, Optional

warnings.filterwarnings('ignore')

print("📊 Banco Insights 2.0 - Metrics Calculations Documentation")
print(f"📅 Notebook executed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("⚠️  IMPORTANT: Using safe data inspection methods only - no direct CSV loading of large files")

📊 Banco Insights 2.0 - Metrics Calculations Documentation
📅 Notebook executed: 2025-08-12 00:22:10
⚠️  IMPORTANT: Using safe data inspection methods only - no direct CSV loading of large files


### Data Structure Analysis (Safe Inspection)

Instead of loading the full consolidated_cleaned.csv, we'll use safe inspection methods to understand the data structure.

In [2]:
def safe_csv_info(file_path: str, nrows: int = 5) -> dict:
    """
    Safely inspect CSV file structure without loading the full file.

    Args:
        file_path: Path to CSV file
        nrows: Number of rows to sample

    Returns:
        Dictionary with file info
    """
    try:
        # Read only first few rows to understand structure
        sample_df = pd.read_csv(file_path, nrows=nrows)

        info = {
            'columns': list(sample_df.columns),
            'shape_sample': sample_df.shape,
            'dtypes': dict(sample_df.dtypes),
            'sample_data': sample_df.head(3).to_dict('records')
        }

        return info

    except Exception as e:
        return {'error': str(e)}

# Safe inspection of data structure
data_path = 'bacen_project_v1/data/consolidated_cleaned.csv'
data_info = safe_csv_info(data_path, nrows=10)

print("📊 Data Structure Analysis:")
print(f"Columns available: {len(data_info.get('columns', []))}")
if 'columns' in data_info:
    print("\nColumn names:")
    for i, col in enumerate(data_info['columns']):
        print(f"  {i+1:2d}. {col}")

📊 Data Structure Analysis:
Columns available: 15

Column names:
   1. TipoInstituicao
   2. CodInst
   3. AnoMes
   4. NomeRelatorio
   5. NumeroRelatorio
   6. Grupo
   7. Conta
   8. NomeColuna
   9. DescricaoColuna
  10. Saldo
  11. AnoMes_M
  12. AnoMes_Q
  13. AnoMes_Y
  14. NomeRelatorio_Grupo_Coluna
  15. NomeInstituicao


### Metric Identifier Mapping

Based on the unique values analysis, we'll create mappings from the concatenated identifiers to clean column names.

In [3]:
# Load the unique values mapping
with open('EDA/unique_values.json', 'r') as f:
    unique_values = json.load(f)

# Key metric identifiers for Banco Insights
CORE_METRIC_IDENTIFIERS = {
    # Balance Sheet Metrics
    'Ativo_Total': 'Resumo_nagroup_Ativo Total',
    'Carteira_de_Credito_Classificada': 'Resumo_nagroup_Carteira de Crédito Classificada',
    'Patrimonio_Liquido': 'Resumo_nagroup_Patrimônio Líquido',
    'Captacoes': 'Resumo_nagroup_Captações',

    # P&L Metrics
    'Lucro_Liquido': 'Resumo_nagroup_Lucro Líquido',
    'Receitas_de_Intermediacao_Financeira': 'Demonstração de Resultado_Resultado de Intermediação Financeira - Receitas de Intermediação Financeira_Receitas de Intermediação Financeira \\n(a) = (a1) + (a2) + (a3) + (a4) + (a5) + (a6)',
    'Despesas_de_Intermediacao_Financeira': 'Demonstração de Resultado_Resultado de Intermediação Financeira - Despesas de Intermediação Financeira_Despesas de Intermediação Financeira \\n(b) = (b1) + (b2) + (b3) + (b4) + (b5)',

    # Regulatory Metrics
    'Indice_de_Basileia': 'Resumo_nagroup_Índice de Basileia',
    'Indice_de_Imobilizacao': 'Resumo_nagroup_Índice de Imobilização',

    # Client Metrics
    'Quantidade_de_Clientes_com_Operacoes_Ativas': 'Carteira de crédito ativa - quantidade de clientes e de operações_nagroup_Quantidade de clientes com operações ativas',
    'Quantidade_de_Operacoes_Ativas': 'Carteira de crédito ativa - quantidade de clientes e de operações_nagroup_Quantidade de operações ativas'
}

print("🎯 Core Metric Identifiers Defined:")
for clean_name, original_id in CORE_METRIC_IDENTIFIERS.items():
    print(f"  • {clean_name}")
    print(f"    ← {original_id[:80]}..." if len(original_id) > 80 else f"    ← {original_id}")
    print()

🎯 Core Metric Identifiers Defined:
  • Ativo_Total
    ← Resumo_nagroup_Ativo Total

  • Carteira_de_Credito_Classificada
    ← Resumo_nagroup_Carteira de Crédito Classificada

  • Patrimonio_Liquido
    ← Resumo_nagroup_Patrimônio Líquido

  • Captacoes
    ← Resumo_nagroup_Captações

  • Lucro_Liquido
    ← Resumo_nagroup_Lucro Líquido

  • Receitas_de_Intermediacao_Financeira
    ← Demonstração de Resultado_Resultado de Intermediação Financeira - Receitas de In...

  • Despesas_de_Intermediacao_Financeira
    ← Demonstração de Resultado_Resultado de Intermediação Financeira - Despesas de In...

  • Indice_de_Basileia
    ← Resumo_nagroup_Índice de Basileia

  • Indice_de_Imobilizacao
    ← Resumo_nagroup_Índice de Imobilização

  • Quantidade_de_Clientes_com_Operacoes_Ativas
    ← Carteira de crédito ativa - quantidade de clientes e de operações_nagroup_Quanti...

  • Quantidade_de_Operacoes_Ativas
    ← Carteira de crédito ativa - quantidade de clientes e de operações_nagroup_Quant

---

## 2. Simple Aggregations {#simple-aggregations}

These are direct value extractions from the BACEN data without complex calculations.

### 2.1 Balance Sheet Aggregations

**Purpose**: Extract key balance sheet metrics for financial analysis

In [4]:
class SimpleAggregations:
    """
    Simple aggregation methods for extracting direct metrics from BACEN data.
    These are 1:1 mappings with no calculations involved.
    """

    @staticmethod
    def extract_balance_sheet_metrics(df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract core balance sheet metrics for each institution-period.

        Args:
            df: DataFrame with consolidated data structure

        Returns:
            DataFrame with balance sheet metrics as columns
        """

        balance_sheet_metrics = {
            'Ativo_Total': 'Resumo_nagroup_Ativo Total',
            'Carteira_de_Credito_Classificada': 'Resumo_nagroup_Carteira de Crédito Classificada',
            'Patrimonio_Liquido': 'Resumo_nagroup_Patrimônio Líquido',
            'Captacoes': 'Resumo_nagroup_Captações',
            'Passivo_Circulante_e_Exigivel_a_Longo_Prazo': 'Resumo_nagroup_Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros'
        }

        # Demonstration of extraction logic (without actual data loading)
        extraction_logic = """
        For each metric in balance_sheet_metrics:
        1. Filter df where NomeRelatorio_Grupo_Coluna == identifier
        2. Select columns: CodInst, NomeInstituicao, AnoMes, AnoMes_Q, Saldo
        3. Rename Saldo to clean metric name
        4. Join all metrics by (CodInst, AnoMes)
        """

        print("📊 Balance Sheet Metrics Extraction:")
        print(extraction_logic)

        # Return sample structure
        sample_result = pd.DataFrame({
            'CodInst': ['12345678', '87654321'],
            'NomeInstituicao': ['Banco Exemplo S.A.', 'Cooperativa Modelo'],
            'AnoMes': ['2024-09-30', '2024-09-30'],
            'AnoMes_Q': ['2024Q3', '2024Q3'],
            'Ativo_Total': [1000000000, 500000000],
            'Patrimonio_Liquido': [100000000, 50000000],
            'Carteira_de_Credito_Classificada': [700000000, 300000000],
            'Captacoes': [800000000, 400000000]
        })

        return sample_result

    @staticmethod
    def validate_balance_sheet_equation(df: pd.DataFrame) -> pd.DataFrame:
        """
        Validate the fundamental balance sheet equation: Assets = Liabilities + Equity

        Args:
            df: DataFrame with balance sheet metrics

        Returns:
            DataFrame with validation results
        """

        # Calculate validation
        df['Balance_Check'] = df['Ativo_Total'] - (df['Passivo_Circulante_e_Exigivel_a_Longo_Prazo'] + df['Patrimonio_Liquido'])
        df['Balance_Valid'] = abs(df['Balance_Check']) < 1000  # Allow R$1k tolerance

        validation_summary = {
            'total_records': len(df),
            'valid_records': df['Balance_Valid'].sum(),
            'validation_rate': df['Balance_Valid'].mean() * 100,
            'max_deviation': df['Balance_Check'].abs().max()
        }

        print("✅ Balance Sheet Validation Results:")
        for key, value in validation_summary.items():
            print(f"  • {key.replace('_', ' ').title()}: {value}")

        return df

# Demonstrate the aggregation logic
sample_bs = SimpleAggregations.extract_balance_sheet_metrics(None)
print("\n📋 Sample Balance Sheet Metrics:")
print(sample_bs)

📊 Balance Sheet Metrics Extraction:

        For each metric in balance_sheet_metrics:
        1. Filter df where NomeRelatorio_Grupo_Coluna == identifier
        2. Select columns: CodInst, NomeInstituicao, AnoMes, AnoMes_Q, Saldo
        3. Rename Saldo to clean metric name
        4. Join all metrics by (CodInst, AnoMes)
        

📋 Sample Balance Sheet Metrics:
    CodInst     NomeInstituicao      AnoMes AnoMes_Q  Ativo_Total  \
0  12345678  Banco Exemplo S.A.  2024-09-30   2024Q3   1000000000   
1  87654321  Cooperativa Modelo  2024-09-30   2024Q3    500000000   

   Patrimonio_Liquido  Carteira_de_Credito_Classificada  Captacoes  
0           100000000                         700000000  800000000  
1            50000000                         300000000  400000000  


### 2.2 Income Statement Aggregations

**Purpose**: Extract P&L components for profitability analysis

In [5]:
def extract_income_statement_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extract core income statement metrics.

    Args:
        df: DataFrame with consolidated data structure

    Returns:
        DataFrame with P&L metrics as columns
    """

    income_statement_metrics = {
        # Revenue Components
        'Receitas_de_Intermediacao_Financeira': 'Demonstração de Resultado_Resultado de Intermediação Financeira - Receitas de Intermediação Financeira_Receitas de Intermediação Financeira \\n(a) = (a1) + (a2) + (a3) + (a4) + (a5) + (a6)',
        'Rendas_de_Operacoes_de_Credito': 'Demonstração de Resultado_Resultado de Intermediação Financeira - Receitas de Intermediação Financeira_Rendas de Operações de Crédito \\n(a1)',
        'Rendas_de_Prestacao_de_Servicos': 'Demonstração de Resultado_Outras Receitas/Despesas Operacionais_Rendas de Prestação de Serviços \\n(d1)',
        'Rendas_de_Tarifas_Bancarias': 'Demonstração de Resultado_Outras Receitas/Despesas Operacionais_Rendas de Tarifas Bancárias \\n(d2)',

        # Expense Components
        'Despesas_de_Intermediacao_Financeira': 'Demonstração de Resultado_Resultado de Intermediação Financeira - Despesas de Intermediação Financeira_Despesas de Intermediação Financeira \\n(b) = (b1) + (b2) + (b3) + (b4) + (b5)',
        'Despesas_de_Captacao': 'Demonstração de Resultado_Resultado de Intermediação Financeira - Despesas de Intermediação Financeira_Despesas de Captação \\n(b1)',
        'Despesas_de_Pessoal': 'Demonstração de Resultado_Outras Receitas/Despesas Operacionais_Despesas de Pessoal \\n(d3)',
        'Despesas_Administrativas': 'Demonstração de Resultado_Outras Receitas/Despesas Operacionais_Despesas Administrativas \\n(d4)',

        # Result Components
        'Resultado_de_Intermediacao_Financeira': 'Demonstração de Resultado_Resultado de Intermediação Financeira_Resultado de Intermediação Financeira \\n(c) = (a) + (b)',
        'Resultado_Operacional': 'Demonstração de Resultado_nagroup_Resultado Operacional \\n(e) = (c) + (d)',
        'Lucro_Liquido': 'Demonstração de Resultado_nagroup_Lucro Líquido \\n(j) = (g) + (h) + (i)'
    }

    print("📈 Income Statement Metrics Mapping:")
    for metric_name, identifier in income_statement_metrics.items():
        print(f"  • {metric_name}")
        # Show shorter version of identifier for readability
        short_id = identifier.split('_')[0] + '...' + identifier.split('\\n')[-1] if '\\n' in identifier else identifier
        print(f"    ← {short_id}\n")

    # Sample result structure
    sample_result = pd.DataFrame({
        'CodInst': ['12345678', '87654321'],
        'AnoMes_Q': ['2024Q3', '2024Q3'],
        'Receitas_de_Intermediacao_Financeira': [50000000, 25000000],
        'Despesas_de_Intermediacao_Financeira': [-30000000, -15000000],
        'Resultado_de_Intermediacao_Financeira': [20000000, 10000000],
        'Despesas_de_Pessoal': [-8000000, -4000000],
        'Despesas_Administrativas': [-5000000, -2500000],
        'Lucro_Liquido': [7000000, 3500000]
    })

    return sample_result

# Demonstrate P&L extraction
sample_pl = extract_income_statement_metrics(None)
print("\n📋 Sample P&L Metrics:")
print(sample_pl)

📈 Income Statement Metrics Mapping:
  • Receitas_de_Intermediacao_Financeira
    ← Demonstração de Resultado...(a) = (a1) + (a2) + (a3) + (a4) + (a5) + (a6)

  • Rendas_de_Operacoes_de_Credito
    ← Demonstração de Resultado...(a1)

  • Rendas_de_Prestacao_de_Servicos
    ← Demonstração de Resultado...(d1)

  • Rendas_de_Tarifas_Bancarias
    ← Demonstração de Resultado...(d2)

  • Despesas_de_Intermediacao_Financeira
    ← Demonstração de Resultado...(b) = (b1) + (b2) + (b3) + (b4) + (b5)

  • Despesas_de_Captacao
    ← Demonstração de Resultado...(b1)

  • Despesas_de_Pessoal
    ← Demonstração de Resultado...(d3)

  • Despesas_Administrativas
    ← Demonstração de Resultado...(d4)

  • Resultado_de_Intermediacao_Financeira
    ← Demonstração de Resultado...(c) = (a) + (b)

  • Resultado_Operacional
    ← Demonstração de Resultado...(e) = (c) + (d)

  • Lucro_Liquido
    ← Demonstração de Resultado...(j) = (g) + (h) + (i)


📋 Sample P&L Metrics:
    CodInst AnoMes_Q  Receitas_de_Inte

### 2.3 Credit Portfolio Aggregations

**Purpose**: Extract credit portfolio breakdowns by modality, risk, and client type

In [6]:
def extract_credit_portfolio_metrics(df: pd.DataFrame) -> dict:
    """
    Extract credit portfolio metrics by different dimensions.

    Returns:
        Dictionary with different credit portfolio views
    """

    credit_portfolio_views = {
        'risk_levels': {
            'AA': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_AA',
            'A': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_A',
            'B': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_B',
            'C': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_C',
            'D': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_D',
            'E': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_E',
            'F': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_F',
            'G': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_G',
            'H': 'Carteira de crédito ativa - por nível de risco da operação_nagroup_H'
        },

        'pf_modalities': {
            'Emprestimo_com_Consignacao': 'Carteira de crédito ativa Pessoa Física - modalidade e prazo de vencimento_Empréstimo com Consignação em Folha_Total',
            'Emprestimo_sem_Consignacao': 'Carteira de crédito ativa Pessoa Física - modalidade e prazo de vencimento_Empréstimo sem Consignação em Folha_Total',
            'Veiculos': 'Carteira de crédito ativa Pessoa Física - modalidade e prazo de vencimento_Veículos_Total',
            'Habitacao': 'Carteira de crédito ativa Pessoa Física - modalidade e prazo de vencimento_Habitação_Total',
            'Cartao_de_Credito': 'Carteira de crédito ativa Pessoa Física - modalidade e prazo de vencimento_Cartão de Crédito_Total',
            'Rural_e_Agroindustrial': 'Carteira de crédito ativa Pessoa Física - modalidade e prazo de vencimento_Rural e Agroindustrial_Total'
        },

        'pj_modalities': {
            'Capital_de_Giro': 'Carteira de crédito ativa Pessoa Jurídica - modalidade e prazo de vencimento_Capital de Giro_Total',
            'Investimento': 'Carteira de crédito ativa Pessoa Jurídica - modalidade e prazo de vencimento_Investimento_Total',
            'Capital_de_Giro_Rotativo': 'Carteira de crédito ativa Pessoa Jurídica - modalidade e prazo de vencimento_Capital de Giro Rotativo_Total',
            'Comercio_Exterior': 'Carteira de crédito ativa Pessoa Jurídica - modalidade e prazo de vencimento_Comércio Exterior_Total',
            'Rural_e_Agroindustrial': 'Carteira de crédito ativa Pessoa Jurídica - modalidade e prazo de vencimento_Rural e Agroindustrial_Total'
        },

        'regions': {
            'Sudeste': 'Carteira de crédito ativa - por região geográfica_nagroup_Sudeste',
            'Sul': 'Carteira de crédito ativa - por região geográfica_nagroup_Sul',
            'Nordeste': 'Carteira de crédito ativa - por região geográfica_nagroup_Nordeste',
            'Norte': 'Carteira de crédito ativa - por região geográfica_nagroup_Norte',
            'Centro_oeste': 'Carteira de crédito ativa - por região geográfica_nagroup_Centro-oeste'
        }
    }

    print("🏦 Credit Portfolio Dimensions:")
    for dimension, metrics in credit_portfolio_views.items():
        print(f"\n📊 {dimension.replace('_', ' ').title()}:")
        for metric_name in metrics.keys():
            print(f"  • {metric_name}")

    # Sample credit risk distribution
    sample_risk_distribution = pd.DataFrame({
        'CodInst': ['12345678'] * 9,
        'AnoMes_Q': ['2024Q3'] * 9,
        'Risk_Level': ['AA', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
        'Credit_Amount': [300000000, 200000000, 150000000, 30000000, 15000000, 3000000, 1500000, 300000, 200000],
        'Credit_Percentage': [42.86, 28.57, 21.43, 4.29, 2.14, 0.43, 0.21, 0.04, 0.03]
    })

    print("\n📋 Sample Credit Risk Distribution:")
    print(sample_risk_distribution)

    return credit_portfolio_views

# Demonstrate credit portfolio extraction
credit_views = extract_credit_portfolio_metrics(None)

🏦 Credit Portfolio Dimensions:

📊 Risk Levels:
  • AA
  • A
  • B
  • C
  • D
  • E
  • F
  • G
  • H

📊 Pf Modalities:
  • Emprestimo_com_Consignacao
  • Emprestimo_sem_Consignacao
  • Veiculos
  • Habitacao
  • Cartao_de_Credito
  • Rural_e_Agroindustrial

📊 Pj Modalities:
  • Capital_de_Giro
  • Investimento
  • Capital_de_Giro_Rotativo
  • Comercio_Exterior
  • Rural_e_Agroindustrial

📊 Regions:
  • Sudeste
  • Sul
  • Nordeste
  • Norte
  • Centro_oeste

📋 Sample Credit Risk Distribution:
    CodInst AnoMes_Q Risk_Level  Credit_Amount  Credit_Percentage
0  12345678   2024Q3         AA      300000000              42.86
1  12345678   2024Q3          A      200000000              28.57
2  12345678   2024Q3          B      150000000              21.43
3  12345678   2024Q3          C       30000000               4.29
4  12345678   2024Q3          D       15000000               2.14
5  12345678   2024Q3          E        3000000               0.43
6  12345678   2024Q3          F        

---

## 3. Complex Financial Ratios {#complex-ratios}

These calculations involve multiple metrics and require mathematical operations.

### 3.1 Profitability Ratios

**Critical for**: Performance analysis, peer benchmarking, and investment decisions

In [7]:
class ProfitabilityRatios:
    """
    Comprehensive profitability ratio calculations following banking industry standards.

    All calculations are annualized and based on average balances where appropriate.
    """

    @staticmethod
    def calculate_roe(net_income_ttm: float, avg_equity: float) -> float:
        """
        Calculate Return on Equity (ROE).

        Formula: ROE = (Net Income TTM / Average Shareholders' Equity) * 100

        Args:
            net_income_ttm: Net income trailing twelve months
            avg_equity: Average shareholders' equity (current + previous period) / 2

        Returns:
            ROE as percentage

        Industry Benchmarks:
            • Excellent: > 15%
            • Good: 10-15%
            • Average: 5-10%
            • Poor: < 5%
        """
        if avg_equity <= 0:
            return np.nan

        roe = (net_income_ttm / avg_equity) * 100
        return round(roe, 2)

    @staticmethod
    def calculate_roa(net_income_ttm: float, avg_assets: float) -> float:
        """
        Calculate Return on Assets (ROA).

        Formula: ROA = (Net Income TTM / Average Total Assets) * 100

        Args:
            net_income_ttm: Net income trailing twelve months
            avg_assets: Average total assets (current + previous period) / 2

        Returns:
            ROA as percentage

        Industry Benchmarks:
            • Excellent: > 1.5%
            • Good: 1.0-1.5%
            • Average: 0.5-1.0%
            • Poor: < 0.5%
        """
        if avg_assets <= 0:
            return np.nan

        roa = (net_income_ttm / avg_assets) * 100
        return round(roa, 2)

    @staticmethod
    def calculate_nim(net_interest_income: float, avg_earning_assets: float) -> float:
        """
        Calculate Net Interest Margin (NIM).

        Formula: NIM = (Net Interest Income / Average Earning Assets) * 100

        Args:
            net_interest_income: Interest income - Interest expense
            avg_earning_assets: Average interest-earning assets

        Returns:
            NIM as percentage

        BACEN Context:
            Net Interest Income ≈ Resultado de Intermediação Financeira
            Earning Assets ≈ Operações de Crédito + TVM + Aplicações Interfinanceiras
        """
        if avg_earning_assets <= 0:
            return np.nan

        nim = (net_interest_income / avg_earning_assets) * 100
        return round(nim, 2)

# Demonstration with sample data
sample_institution_data = {
    'institution_name': 'Banco Exemplo S.A.',
    'period': '2024Q3',
    'net_income_ttm': 28000000,  # R$ 28 million TTM
    'current_equity': 250000000,  # R$ 250 million
    'previous_equity': 240000000,  # R$ 240 million
    'current_assets': 2000000000,  # R$ 2 billion
    'previous_assets': 1900000000,  # R$ 1.9 billion
    'net_interest_income': 80000000,  # R$ 80 million
    'earning_assets': 1600000000  # R$ 1.6 billion
}

# Calculate ratios
avg_equity = (sample_institution_data['current_equity'] + sample_institution_data['previous_equity']) / 2
avg_assets = (sample_institution_data['current_assets'] + sample_institution_data['previous_assets']) / 2

roe = ProfitabilityRatios.calculate_roe(sample_institution_data['net_income_ttm'], avg_equity)
roa = ProfitabilityRatios.calculate_roa(sample_institution_data['net_income_ttm'], avg_assets)
nim = ProfitabilityRatios.calculate_nim(sample_institution_data['net_interest_income'], sample_institution_data['earning_assets'])

print("🎯 Profitability Ratio Calculations Example:")
print(f"Institution: {sample_institution_data['institution_name']}")
print(f"Period: {sample_institution_data['period']}")
print("\n📊 Results:")
print(f"  • ROE (Return on Equity): {roe}%")
print(f"  • ROA (Return on Assets): {roa}%")
print(f"  • NIM (Net Interest Margin): {nim}%")

print("\n🔍 Calculation Details:")
print(f"  • Net Income TTM: R$ {sample_institution_data['net_income_ttm']:,}")
print(f"  • Average Equity: R$ {avg_equity:,.0f}")
print(f"  • Average Assets: R$ {avg_assets:,.0f}")
print(f"  • ROE Formula: ({sample_institution_data['net_income_ttm']:,} / {avg_equity:,.0f}) × 100 = {roe}%")

🎯 Profitability Ratio Calculations Example:
Institution: Banco Exemplo S.A.
Period: 2024Q3

📊 Results:
  • ROE (Return on Equity): 11.43%
  • ROA (Return on Assets): 1.44%
  • NIM (Net Interest Margin): 5.0%

🔍 Calculation Details:
  • Net Income TTM: R$ 28,000,000
  • Average Equity: R$ 245,000,000
  • Average Assets: R$ 1,950,000,000
  • ROE Formula: (28,000,000 / 245,000,000) × 100 = 11.43%


### 3.2 Regulatory Capital Ratios (Basel Framework)

**Critical for**: Regulatory compliance, risk management, and capital planning

In [8]:
class RegulatoryCapitalRatios:
    """
    Basel framework capital ratio calculations following BACEN regulations.

    Key Regulatory Requirements (Brazil):
    • Basel Ratio: Minimum 8%
    • Tier 1 Capital Ratio: Minimum 6%
    • Common Equity Tier 1: Minimum 4.5%
    """

    @staticmethod
    def calculate_basel_ratio(regulatory_capital: float, rwa_total: float) -> float:
        """
        Calculate Basel Capital Adequacy Ratio.

        Formula: Basel Ratio = (Regulatory Capital / Risk-Weighted Assets) * 100

        Args:
            regulatory_capital: Total regulatory capital (Patrimônio de Referência)
            rwa_total: Total risk-weighted assets (Ativos Ponderados pelo Risco)

        Returns:
            Basel ratio as percentage

        BACEN Mapping:
            • Regulatory Capital = "Patrimônio de Referência para Comparação com o RWA"
            • RWA Total = "Ativos Ponderados pelo Risco (RWA)"
        """
        if rwa_total <= 0:
            return np.nan

        basel_ratio = (regulatory_capital / rwa_total) * 100
        return round(basel_ratio, 2)

    @staticmethod
    def calculate_tier1_ratio(tier1_capital: float, rwa_total: float) -> float:
        """
        Calculate Tier 1 Capital Ratio.

        Formula: Tier 1 Ratio = (Tier 1 Capital / Risk-Weighted Assets) * 100

        Args:
            tier1_capital: Tier 1 regulatory capital
            rwa_total: Total risk-weighted assets

        Returns:
            Tier 1 ratio as percentage

        BACEN Mapping:
            • Tier 1 Capital = "Patrimônio de Referência Nível I para Comparação com RWA"
        """
        if rwa_total <= 0:
            return np.nan

        tier1_ratio = (tier1_capital / rwa_total) * 100
        return round(tier1_ratio, 2)

    @staticmethod
    def calculate_common_equity_tier1_ratio(cet1_capital: float, rwa_total: float) -> float:
        """
        Calculate Common Equity Tier 1 (CET1) Ratio.

        Formula: CET1 Ratio = (CET1 Capital / Risk-Weighted Assets) * 100

        Args:
            cet1_capital: Common Equity Tier 1 capital
            rwa_total: Total risk-weighted assets

        Returns:
            CET1 ratio as percentage

        BACEN Mapping:
            • CET1 Capital = "Capital Principal para Comparação com RWA"
        """
        if rwa_total <= 0:
            return np.nan

        cet1_ratio = (cet1_capital / rwa_total) * 100
        return round(cet1_ratio, 2)

    @staticmethod
    def assess_capital_adequacy(basel_ratio: float, tier1_ratio: float, cet1_ratio: float) -> dict:
        """
        Assess capital adequacy against regulatory minimums.

        Args:
            basel_ratio: Basel capital ratio
            tier1_ratio: Tier 1 capital ratio
            cet1_ratio: CET1 capital ratio

        Returns:
            Dictionary with adequacy assessment
        """

        REGULATORY_MINIMUMS = {
            'basel_minimum': 8.0,
            'tier1_minimum': 6.0,
            'cet1_minimum': 4.5
        }

        assessment = {
            'basel_adequate': basel_ratio >= REGULATORY_MINIMUMS['basel_minimum'],
            'tier1_adequate': tier1_ratio >= REGULATORY_MINIMUMS['tier1_minimum'],
            'cet1_adequate': cet1_ratio >= REGULATORY_MINIMUMS['cet1_minimum'],
            'overall_adequate': all([
                basel_ratio >= REGULATORY_MINIMUMS['basel_minimum'],
                tier1_ratio >= REGULATORY_MINIMUMS['tier1_minimum'],
                cet1_ratio >= REGULATORY_MINIMUMS['cet1_minimum']
            ]),
            'basel_buffer': basel_ratio - REGULATORY_MINIMUMS['basel_minimum'],
            'tier1_buffer': tier1_ratio - REGULATORY_MINIMUMS['tier1_minimum'],
            'cet1_buffer': cet1_ratio - REGULATORY_MINIMUMS['cet1_minimum']
        }

        return assessment

# Sample regulatory capital data
sample_capital_data = {
    'institution_name': 'Banco Exemplo S.A.',
    'period': '2024Q3',
    'regulatory_capital': 180000000,  # R$ 180 million
    'tier1_capital': 150000000,      # R$ 150 million
    'cet1_capital': 120000000,       # R$ 120 million
    'rwa_total': 1200000000,         # R$ 1.2 billion
    'rwa_credit': 1000000000,        # R$ 1.0 billion
    'rwa_market': 100000000,         # R$ 100 million
    'rwa_operational': 100000000     # R$ 100 million
}

# Calculate regulatory ratios
basel_ratio = RegulatoryCapitalRatios.calculate_basel_ratio(
    sample_capital_data['regulatory_capital'],
    sample_capital_data['rwa_total']
)

tier1_ratio = RegulatoryCapitalRatios.calculate_tier1_ratio(
    sample_capital_data['tier1_capital'],
    sample_capital_data['rwa_total']
)

cet1_ratio = RegulatoryCapitalRatios.calculate_common_equity_tier1_ratio(
    sample_capital_data['cet1_capital'],
    sample_capital_data['rwa_total']
)

adequacy_assessment = RegulatoryCapitalRatios.assess_capital_adequacy(basel_ratio, tier1_ratio, cet1_ratio)

print("🏛️ Regulatory Capital Ratios Example:")
print(f"Institution: {sample_capital_data['institution_name']}")
print(f"Period: {sample_capital_data['period']}")

print("\n📊 Capital Ratios:")
print(f"  • Basel Ratio: {basel_ratio}% (Minimum: 8.0%)")
print(f"  • Tier 1 Ratio: {tier1_ratio}% (Minimum: 6.0%)")
print(f"  • CET1 Ratio: {cet1_ratio}% (Minimum: 4.5%)")

print("\n✅ Regulatory Adequacy:")
print(f"  • Basel Adequate: {'Yes' if adequacy_assessment['basel_adequate'] else 'No'} (Buffer: +{adequacy_assessment['basel_buffer']:.1f}pp)")
print(f"  • Tier 1 Adequate: {'Yes' if adequacy_assessment['tier1_adequate'] else 'No'} (Buffer: +{adequacy_assessment['tier1_buffer']:.1f}pp)")
print(f"  • CET1 Adequate: {'Yes' if adequacy_assessment['cet1_adequate'] else 'No'} (Buffer: +{adequacy_assessment['cet1_buffer']:.1f}pp)")
print(f"  • Overall Adequate: {'Yes' if adequacy_assessment['overall_adequate'] else 'No'}")

🏛️ Regulatory Capital Ratios Example:
Institution: Banco Exemplo S.A.
Period: 2024Q3

📊 Capital Ratios:
  • Basel Ratio: 15.0% (Minimum: 8.0%)
  • Tier 1 Ratio: 12.5% (Minimum: 6.0%)
  • CET1 Ratio: 10.0% (Minimum: 4.5%)

✅ Regulatory Adequacy:
  • Basel Adequate: Yes (Buffer: +7.0pp)
  • Tier 1 Adequate: Yes (Buffer: +6.5pp)
  • CET1 Adequate: Yes (Buffer: +5.5pp)
  • Overall Adequate: Yes


### 3.3 Efficiency and Asset Quality Ratios

**Critical for**: Operational efficiency analysis and risk assessment

In [9]:
class EfficiencyAndQualityRatios:
    """
    Operational efficiency and asset quality ratio calculations.
    """

    @staticmethod
    def calculate_cost_income_ratio(operating_expenses: float, operating_income: float) -> float:
        """
        Calculate Cost-to-Income Ratio.

        Formula: Cost/Income = (Operating Expenses / Operating Income) * 100

        Args:
            operating_expenses: Personnel + Administrative expenses
            operating_income: Total operating revenues

        Returns:
            Cost/Income ratio as percentage (lower is better)

        BACEN Mapping:
            • Operating Expenses = Despesas de Pessoal + Despesas Administrativas
            • Operating Income = Receitas de Intermediação + Rendas de Serviços + Tarifas
        """
        if operating_income <= 0:
            return np.nan

        cost_income_ratio = (operating_expenses / operating_income) * 100
        return round(cost_income_ratio, 2)

    @staticmethod
    def calculate_credit_loss_ratio(provisions: float, gross_credit: float) -> float:
        """
        Calculate Credit Loss Ratio.

        Formula: Credit Loss = (Provisions for Credit Losses / Gross Credit Portfolio) * 100

        Args:
            provisions: Credit loss provisions
            gross_credit: Gross credit portfolio

        Returns:
            Credit loss ratio as percentage

        BACEN Mapping:
            • Provisions = "Resultado de Provisão para Créditos de Difícil Liquidação"
            • Gross Credit = "Operações de Crédito" (before provisions)
        """
        if gross_credit <= 0:
            return np.nan

        credit_loss_ratio = abs(provisions / gross_credit) * 100
        return round(credit_loss_ratio, 2)

    @staticmethod
    def calculate_asset_turnover(operating_income: float, avg_assets: float) -> float:
        """
        Calculate Asset Turnover Ratio.

        Formula: Asset Turnover = Operating Income / Average Assets

        Args:
            operating_income: Total operating income
            avg_assets: Average total assets

        Returns:
            Asset turnover ratio (higher is better)
        """
        if avg_assets <= 0:
            return np.nan

        asset_turnover = operating_income / avg_assets
        return round(asset_turnover, 4)

# Sample efficiency data
sample_efficiency_data = {
    'institution_name': 'Banco Exemplo S.A.',
    'period': '2024Q3',
    'personnel_expenses': -25000000,     # R$ -25 million
    'administrative_expenses': -15000000, # R$ -15 million
    'operating_income': 120000000,       # R$ 120 million
    'credit_provisions': -8000000,       # R$ -8 million
    'gross_credit': 800000000,           # R$ 800 million
    'avg_assets': 2000000000             # R$ 2 billion
}

# Calculate efficiency ratios
operating_expenses = abs(sample_efficiency_data['personnel_expenses']) + abs(sample_efficiency_data['administrative_expenses'])

cost_income_ratio = EfficiencyAndQualityRatios.calculate_cost_income_ratio(
    operating_expenses,
    sample_efficiency_data['operating_income']
)

credit_loss_ratio = EfficiencyAndQualityRatios.calculate_credit_loss_ratio(
    sample_efficiency_data['credit_provisions'],
    sample_efficiency_data['gross_credit']
)

asset_turnover = EfficiencyAndQualityRatios.calculate_asset_turnover(
    sample_efficiency_data['operating_income'],
    sample_efficiency_data['avg_assets']
)

print("⚙️ Efficiency and Quality Ratios Example:")
print(f"Institution: {sample_efficiency_data['institution_name']}")
print(f"Period: {sample_efficiency_data['period']}")

print("\n📊 Efficiency Ratios:")
print(f"  • Cost-to-Income Ratio: {cost_income_ratio}%")
print(f"  • Credit Loss Ratio: {credit_loss_ratio}%")
print(f"  • Asset Turnover: {asset_turnover}x")

print("\n🎯 Interpretation:")
efficiency_grade = "Good" if cost_income_ratio < 50 else "Average" if cost_income_ratio < 60 else "Poor"
credit_quality = "Good" if credit_loss_ratio < 1.5 else "Average" if credit_loss_ratio < 3.0 else "Poor"
print(f"  • Cost Efficiency: {efficiency_grade} (Target: < 50%)")
print(f"  • Credit Quality: {credit_quality} (Target: < 1.5%)")
print(f"  • Asset Utilization: {asset_turnover}x revenue per R$1 of assets")

⚙️ Efficiency and Quality Ratios Example:
Institution: Banco Exemplo S.A.
Period: 2024Q3

📊 Efficiency Ratios:
  • Cost-to-Income Ratio: 33.33%
  • Credit Loss Ratio: 1.0%
  • Asset Turnover: 0.06x

🎯 Interpretation:
  • Cost Efficiency: Good (Target: < 50%)
  • Credit Quality: Good (Target: < 1.5%)
  • Asset Utilization: 0.06x revenue per R$1 of assets
